In [4]:
from __future__ import absolute_import, division, print_function

import argparse
import glob
import logging
import os
import random

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler

try:
    from torch.utils.tensorboard import SummaryWriter
except:
    from tensorboardX import SummaryWriter

from tqdm import tqdm, trange

from transformers import (WEIGHTS_NAME, BertConfig,
                                  BertForSequenceClassification, BertTokenizer,
                                  RobertaConfig,
                                  RobertaForSequenceClassification,
                                  RobertaTokenizer,
                                  XLMConfig, XLMForSequenceClassification,
                                  XLMTokenizer, XLNetConfig,
                                  XLNetForSequenceClassification,
                                  XLNetTokenizer,
                                  DistilBertConfig,
                                  DistilBertForSequenceClassification,
                                  DistilBertTokenizer)

from transformers import AdamW, WarmupLinearSchedule

from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features

I1023 10:45:22.894406 140680930260800 file_utils.py:39] PyTorch version 1.1.0 available.
I1023 10:45:23.145264 140680930260800 modeling_xlnet.py:194] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [9]:
processor = processors['SST-2'.lower()]()
# args.output_mode = output_modes['SST-2']
label_list = processor.get_labels()
num_labels = len(label_list)

In [8]:
processor

In [ ]:
examples = processor.get_dev_examples('/home/raghu/transformers/data/glue_data/SST-2/') if evaluate else processor.get_train_examples('/home/raghu/transformers/data/glue_data/SST-2/')

In [14]:
tr=processor.get_train_examples('/home/raghu/transformers/data/sentence_classification/sentiment/')

In [19]:
# loading model for inference

In [16]:
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    'distilbert': (DistilBertConfig, DistilBertForSequenceClassification, DistilBertTokenizer)
}

In [17]:
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']

In [20]:
model = model_class.from_pretrained('/home/raghu/sentiment_bert/')
tokenizer = tokenizer_class.from_pretrained('/home/raghu/sentiment_bert/', do_lower_case=True)

I1023 11:42:14.330874 140680930260800 configuration_utils.py:148] loading configuration file /home/raghu/sentiment_bert/config.json
I1023 11:42:14.333612 140680930260800 configuration_utils.py:168] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": "sst-2",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "output_past": true,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "use_bfloat16": false,
  "vocab_size": 30522
}

I1023 11:42:14.336176 140680930260800 modeling_utils.py:334] loading weights file /home/raghu/sentiment_bert/pytorch_model.bin
I1023 11:42:17.498933 140680930260800 tokenization_utils.py:306] Model name '/home/raghu/sentiment_bert/' not found

In [21]:
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_aff

In [45]:
codes_defined = {0:'Negative',1:'Positive'}

In [50]:
def get_features(text,tokenizer,label_list=['0','1']):
    
    '''
    parms: label_list : list of labels in strings ['0','1'], default set for sentiment
           tokenzier
           examples: list of single example for inference
    
    '''
    
    ex = {
          "guid": "train-1",
          "label": "0",
          "text_a": text,
          "text_b": null
    }
    examples=[ex]
    features = convert_examples_to_features(examples,
                                            tokenizer,
                                            label_list=label_list,
                                            max_length=128,
                                            output_mode="classification",
                                            pad_on_left=False,                 # pad on the left for xlnet
                                            pad_token=tokenizer.convert_tokens_to_ids([tokenizer.pad_token])[0],
                                            pad_token_segment_id=0,
            )
    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
    all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
    eval_dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)
    return eval_dataset
def get_predictions(batch):
    with torch.no_grad():
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[3]}
        outputs = model(**inputs)
        tmp_eval_loss, logits = outputs[:2]
    preds = logits.detach().cpu().numpy()
    return np.argmax(preds, axis=1)

In [48]:
for i in text:
    eval_dataset=get_features(i,tokenizer)
    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=1)
    print('Prediction: ',codes_defined[[get_predictions(tuple(t for t in x)) for x in eval_dataloader][0][0]])

I1023 12:48:17.599095 140680930260800 glue.py:80] Writing example 0
I1023 12:48:17.601741 140680930260800 glue.py:119] *** Example ***
I1023 12:48:17.603106 140680930260800 glue.py:120] guid: train-1
I1023 12:48:17.604523 140680930260800 glue.py:121] input_ids: 101 1045 3328 1016 2000 1017 2463 2015 16215 17599 1999 1037 2733 1998 2043 2131 2000 1996 15373 2391 2027 2360 4316 2022 2025 2800 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I1023 12:48:17.605798 140680930260800 glue.py:122] attention_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
I1023 12:48:17.606992 140680930260800 glue.py:123] token_type_ids: 0 0 0 0 0 

Prediction:  Negative


In [58]:
type(tr[0].text_b)

NoneType

In [59]:
NoneType

NameError: name 'NoneType' is not defined